In [45]:
from create_dist_matrix import create_dist_matrix, create_dist_mat_2
from get_avail_act import get_avail_act
from get_best_action import get_best_action
from get_or_tools_sol import or_solution
from get_random_traj import get_random_traj
from train import train_model
import numpy as np
#Definitions
n_dest = 20 # Set number of destinations
dist_mat = create_dist_matrix(n_dim = n_dest, opt = 1) # Create distance matrix, opt = 0 is random, opt = 2 is fixed example
#dist_mat = create_dist_mat_2() # Use googles example

def tsp_solver(dist_mat,alpha=0.2,gamma=0.8):
    #alpha is the learning rate
    #gamma is the discount factor
    n_dest = dist_mat.shape[0]
    # Train RL model
    q = train_model(dist_mat, n_train = 2000, gamma = gamma, alpha = alpha)# Get trained transition matrix

    #print(q)

    # Use model to find optimum trajectory
    state = [0]
    distance_travel = 0.
    posible_actions = get_avail_act(state, n_dest)
    while posible_actions: # until all destinations are visited
        action = get_best_action(state[-1], posible_actions, q)
        distance_travel += dist_mat[state[-1], action]
        state.append(action)
        posible_actions = get_avail_act(state, n_dest)

    #Back to warehouse
    action = 0
    distance_travel += dist_mat[state[-1], action]
    state.append(action)

    # Get Best optimization possible
    #print("\nGoogle Results: ")
    best_dist, google_route = or_solution(dist_mat)

    # Get random tour
    random_dist, random_route = get_random_traj(dist_mat)

    #Out RL results
    traj =' -> '.join([str(b) for b in state])
    #print(f"Best trajectory found with RL: \n {traj}" )
    #print(f"Total distance travelled with this traj: {distance_travel}\n")
    slow_pctg = 100*(-1+distance_travel/best_dist)
    random_pctg = 100*(-1+distance_travel/random_dist)
    return slow_pctg, traj, distance_travel, google_route, best_dist
    #print(f"RL solution is {100*(-1+distance_travel/best_dist)}% slower than google's solution")

best_pctg = 100
alpha = 0.012
gamma = 0.4
#for alpha in np.linspace(0.012,0.012,1):
#    for gamma in np.linspace(0.4 ,0.4,100):
for _ in range(20):
        slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(dist_mat, alpha=alpha, gamma=gamma)
        if slow_pctg < best_pctg:
            best_pctg = slow_pctg
            if slow_pctg < 0:
                print(f"\nBest solution so far with parameters alpha:{alpha}, gamma:{gamma}, is {-np.around(slow_pctg,decimals=1)}% FASTER than google's solution")
                print(f"RL route:     {rl_route}; distance: {rl_dist}")
                print(f"Google route {google_route}; distance: {google_dist}\n")
            else:
                print(f"\nBest solution so far with parameters alpha:{alpha}, gamma:{gamma}, is {np.around(slow_pctg,decimals=1)}% slower than google's solution")



Best solution so far with parameters alpha:0.012, gamma:0.4, is 0.0% slower than google's solution


In [154]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
import haversine as hs
from scipy.spatial.distance import cdist


In [66]:
import pandas as pd

data_csv = pd.read_csv("clean_data.csv")
df = pd.DataFrame(data_csv)


In [67]:
df.drop_duplicates(subset=['CourierId','Latitude','Longitude'],keep='first',inplace=True)

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
3,324293,55.631359,12.656990
8,324293,55.638429,12.634550
12,324293,NaN,NaN
15,324293,55.656790,12.604109
...,...,...,...
254923,0,55.569114,12.618886
254924,0,55.568978,12.619223
254925,0,55.568047,12.618471
254926,0,55.570509,12.620062


In [68]:
df = df.dropna(axis=0)
index_list = [i for i in range(len(df))]
df = df.set_index([index_list])

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
1,324293,55.631359,12.656990
2,324293,55.638429,12.634550
3,324293,55.656790,12.604109
4,324293,55.657004,12.604765
...,...,...,...
67658,0,55.569114,12.618886
67659,0,55.568978,12.619223
67660,0,55.568047,12.618471
67661,0,55.570509,12.620062


In [144]:
df1 = df.loc[df.CourierId == 315408]
df1 = df1.set_index([[i for i in range(len(df1))]])
df1

,CourierId,Latitude,Longitude,Distance
0,315408,55.613332,12.580179,0
1,315408,55.613341,12.579878,0
2,315408,55.613634,12.579380,0
3,315408,55.613924,12.580217,0
4,315408,55.612915,12.579362,0
...,...,...,...,...
1299,315408,55.595589,12.583007,0
1300,315408,55.594216,12.587852,0
1301,315408,55.593754,12.587334,0
1302,315408,55.592284,12.589010,0


In [78]:
df1.Latitude[5]

KeyError: 5

In [125]:
import math

def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians

    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    # Compute spherical distance from spherical coordinates.

    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    cos1 = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + math.cos(phi1)*math.cos(phi2))
    arc = math.acos(cos1)

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc * 6371

print(distance_on_unit_sphere(55.605452, 12.583987, 55.631359, 12.656990))
hs.haversine((55.605452, 12.583987),(55.631359, 12.656990))

5.414025447876791


5.4140329260557705

In [146]:
my_mat = [[0]*len(df1) for _ in range(len(df1))]

for i in df1.index:
    for j in df1.index:
        print(i,j)
        loc_1=(df1.Latitude[i], df1.Longitude[i])
        loc_2=(df1.Latitude[j], df1.Longitude[j])
        my_mat[i][j] = hs.haversine(loc_1,loc_2)

my_mat = np.array(my_mat)
my_mat = my_mat.astype(int)
my_mat


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


KeyboardInterrupt: 

In [72]:
df1

,CourierId,0,1,2,3,4,306,307,308,309,310,311,312,313,314,315,316,317,318
0,324293,0.000000,0.077464,0.060366,0.055141,0.055582,0.070116,0.089870,0.100447,0.078897,0.068708,0.071881,0.075053,0.075539,0.103146,0.073392,0.096022,0.090825,0.107742
1,324293,0.077464,0.000000,0.023527,0.058678,0.058182,0.102273,0.144950,0.130915,0.100427,0.122458,0.120774,0.124458,0.125260,0.166008,0.131523,0.125810,0.156047,0.170665
2,324293,0.060366,0.023527,0.000000,0.035550,0.035102,0.078924,0.121509,0.108112,0.077762,0.099095,0.097275,0.100955,0.101760,0.142900,0.108314,0.102984,0.133241,0.147538
3,324293,0.055141,0.058678,0.035550,0.000000,0.000690,0.043677,0.088649,0.072599,0.042381,0.067180,0.064057,0.067629,0.068467,0.111798,0.077211,0.067465,0.103603,0.116295
4,324293,0.055582,0.058182,0.035102,0.000690,0.000000,0.044215,0.089286,0.073023,0.042764,0.067840,0.064691,0.068260,0.069098,0.112461,0.077880,0.067891,0.104281,0.116956
306,324293,0.070116,0.102273,0.078924,0.043677,0.044215,0.000000,0.048349,0.031692,0.013270,0.031362,0.025073,0.027851,0.028679,0.073616,0.041760,0.026784,0.068489,0.077698
307,324293,0.089870,0.144950,0.121509,0.088649,0.089286,0.048349,0.000000,0.044037,0.058563,0.022799,0.024602,0.021102,0.020253,0.025860,0.016607,0.044625,0.025183,0.029590
308,324293,0.100447,0.130915,0.108112,0.072599,0.073023,0.031692,0.044037,0.000000,0.030522,0.042712,0.035297,0.034547,0.034798,0.068404,0.048921,0.005139,0.069150,0.071136
309,324293,0.078897,0.100427,0.077762,0.042381,0.042764,0.013270,0.058563,0.030522,0.000000,0.043951,0.037057,0.039307,0.040079,0.084322,0.054084,0.025454,0.080318,0.088153
310,324293,0.068708,0.122458,0.099095,0.067180,0.067840,0.031362,0.022799,0.042712,0.043951,0.000000,0.007782,0.008329,0.008317,0.044641,0.010568,0.040624,0.037609,0.049117


In [47]:
mat.shape

(18, 18)

In [58]:
matrice = np.array([[0,0.1,1],[3,0,6],[2,4,0]])

In [104]:
from create_dist_matrix import create_dist_matrix, create_dist_mat_2
from get_avail_act import get_avail_act
from get_best_action import get_best_action
from get_or_tools_sol import or_solution
from get_random_traj import get_random_traj
from train import train_model
import numpy as np
#Definitions
n_dest = 20 # Set number of destinations
#dist_mat = create_dist_matrix(n_dim = n_dest, opt = 2) # Create distance matrix, opt = 0 is random, opt = 2 is fixed example
#dist_mat = create_dist_mat_2() # Use googles example

def tsp_solver(dist_mat,alpha=0.2,gamma=0.8):
    #alpha is the learning rate
    #gamma is the discount factor
    n_dest = dist_mat.shape[0]
    # Train RL model
    q = train_model(dist_mat, n_train = 2000, gamma = gamma, alpha = alpha)# Get trained transition matrix

    #print(q)

    # Use model to find optimum trajectory
    state = [0]
    distance_travel = 0.
    posible_actions = get_avail_act(state, n_dest)
    while posible_actions: # until all destinations are visited
        action = get_best_action(state[-1], posible_actions, q)
        distance_travel += dist_mat[state[-1], action]
        state.append(action)
        posible_actions = get_avail_act(state, n_dest)

    #Back to warehouse
    action = 0
    distance_travel += dist_mat[state[-1], action]
    state.append(action)

    # Get Best optimization possible
    #print("\nGoogle Results: ")
    best_dist, google_route = or_solution(dist_mat)

    # Get random tour
    random_dist, random_route = get_random_traj(dist_mat)

    #Out RL results
    traj =' -> '.join([str(b) for b in state])
    #print(f"Best trajectory found with RL: \n {traj}" )
    #print(f"Total distance travelled with this traj: {distance_travel}\n")
    slow_pctg = 100*(-1+distance_travel/best_dist)
    random_pctg = 100*(-1+distance_travel/random_dist)
    return slow_pctg, traj, distance_travel, google_route, best_dist
    #print(f"RL solution is {100*(-1+distance_travel/best_dist)}% slower than google's solution")

best_pctg = 100
alpha = 0.012
gamma = 0.4
#for alpha in np.linspace(0.012,0.012,1):
#    for gamma in np.linspace(0.4 ,0.4,100):
for _ in range(15):
        slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(my_mat, alpha=alpha, gamma=gamma)
        if slow_pctg < best_pctg:
            print("test")
            best_pctg = slow_pctg
            if slow_pctg < 0:
                print(f"\nBest solution so far with parameters alpha:{alpha}, gamma:{gamma}, is {-np.around(slow_pctg,decimals=1)}% FASTER than google's solution")
                print(f"RL route:     {rl_route}; distance: {rl_dist}")
                print(f"Google route {google_route}; distance: {google_dist}\n")
            else:
                print(f"\nBest solution so far with parameters alpha:{alpha}, gamma:{gamma}, is {np.around(slow_pctg,decimals=1)}% slower than google's solution")
                print(f"RL route:     {rl_route}; distance: {rl_dist}")
                print(f"Google route {google_route}; distance: {google_dist}\n")

test

Best solution so far with parameters alpha:0.5, gamma:0.5, is 59.1% slower than google's solution
RL route:     0 -> 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 0; distance: 35.0
Google route  0 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 17 -> 3 -> 4 -> 15 -> 16 -> 7 -> 2 -> 1 -> 5 -> 6 -> 0; distance: 22



KeyboardInterrupt: 

In [102]:
slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(my_mat, alpha=alpha, gamma=gamma)

print(rl_dist)

c:\Users\21269\Desktop\RL project\get_immed_reward.py:2: RuntimeWarning: divide by zero encountered in true_divide
  rew = 1./ dist_mat[state[-1],action]
c:\Users\21269\Desktop\RL project\update_q.py:7: RuntimeWarning: invalid value encountered in double_scalars
  q[state[-1],action] += alpha * (imm_reward + gamma * delayed_reward - q[state[-1],action])


35.0


"""# traverse the data
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')
code = address.get('country_code')
zipcode = address.get('postcode')
print('City : ', city)
print('State : ', state)
print('Country : ', country)
print('Zip Code : ', zipcode)"""

In [128]:
distance_list = [0] * len(df)
df["Distance"] = distance_list

df323080

,CourierId,Latitude,Longitude,Distance
0,324293,55.605452,12.583987,0
1,324293,55.631359,12.656990,0
2,324293,55.638429,12.634550,0
3,324293,55.656790,12.604109,0
4,324293,55.657004,12.604765,0
...,...,...,...,...
67658,0,55.569114,12.618886,0
67659,0,55.568978,12.619223,0
67660,0,55.568047,12.618471,0
67661,0,55.570509,12.620062,0


In [162]:
h = df.loc[df.CourierId == 0]
arr = [[k,l] for k,l in zip(h.Latitude,h.Longitude)]
o = cdist(arr,arr)
slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(o, alpha=alpha, gamma=gamma)

KeyboardInterrupt: 

In [159]:

for i in set(df.CourierId):
    print(i)
    new_df = df.loc[df.CourierId == i]
    new_df = new_df.set_index([[i for i in range(len(new_df))]])
    #dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

    """for j in new_df.index:
        for k in new_df.index:
            print("done",i,j,k)
            loc1=(new_df.Latitude[j], new_df.Longitude[j])
            loc2=(new_df.Latitude[k], new_df.Longitude[k])
            dist_matrix[j][k] = hs.haversine(loc1,loc2)"""
    arr = [[k,l] for k,l in zip(new_df.Latitude,new_df.Longitude)]
    dist_matrix = cdist(arr,arr)
    #dist_matrix = my_mat.astype(int)

    slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(dist_matrix, alpha=alpha, gamma=gamma)
    print("reach")
    for idx in df.index:
        if df.CourierId[idx] == i:
            df.Distance[idx] = rl_dist

df

0


KeyboardInterrupt: 

In [157]:
from scipy.spatial.distance import cdist

ENV_SIZE = 10
N_STOPS = 100

# Creating the stops using numpy random points generator
xy = np.random.rand(N_STOPS,2)*ENV_SIZE

# Computing the distances between each points
# Here use euclidean distances, but any metric would do
# This distance matrix can actually represent a time, a distance or something else
distance_matrix = cdist(xy,xy)
xy

array([[3.86720192, 5.24967896],
       [6.41316922, 0.81735344],
       [9.08371871, 3.5797291 ],
       [1.8633534 , 1.74582271],
       [0.63276242, 9.02389088],
       [7.53304709, 6.93023763],
       [9.43074531, 1.55456606],
       [7.94045598, 3.10760419],
       [2.59621871, 1.24961026],
       [6.58695436, 0.48700354],
       [6.31024207, 2.82693173],
       [7.31852499, 2.91009116],
       [1.81809317, 1.0374901 ],
       [9.94963102, 9.96867959],
       [2.30431891, 7.2392988 ],
       [5.40669428, 6.69005678],
       [3.53800473, 0.97321346],
       [1.22051403, 9.22319772],
       [2.04857168, 2.16583188],
       [4.89684612, 2.43436635],
       [4.56445032, 6.72176783],
       [1.11751294, 0.66456881],
       [3.24951457, 7.29648156],
       [3.9654942 , 4.50790419],
       [1.44200184, 3.56737749],
       [1.80252861, 8.76572537],
       [8.91304562, 7.57237894],
       [2.37314362, 0.04546939],
       [1.8383833 , 9.05550752],
       [0.6443261 , 5.54327919],
       [0.

In [163]:
class DeliveryEnvironment:

    def reset(self):
        """Restart the environment for experience replay
        Returns the first state
        """
        pass

    def step(self,a):
        """Takes an action in a given state
        Returns:
            s_next: the next state
            reward: the reward for such action
            done: if the simulation is done
        """
        pass

    def render(self):
        """Visualize the environment state
        """
        pass

In [164]:
from scipy.spatial.distance import cdist
Q = cdist(xy,xy)

In [166]:
class QAgent():
    def __init__(self,states_size,actions_size,epsilon = 1.0,
    epsilon_min = 0.01,epsilon_decay = 0.999,gamma = 0.95,lr = 0.8):
        self.states_size = states_size
        self.actions_size = actions_size
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.lr = lr
        self.Q = self.build_model(states_size,actions_size)


    def build_model(self,states_size,actions_size):
        Q = np.zeros([states_size,actions_size])
        return Q


    def train(self,s,a,r,s_next):
        self.Q[s,a] = self.Q[s,a] + self.lr * (r + self.gamma*np.max(self.Q[s_next,a]) - self.Q[s,a])

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def act(self,s):

        q = self.Q[s,:]

        if np.random.rand() > self.epsilon:
            a = np.argmax(q)
        else:
            a = np.random.randint(self.actions_size)

        return a

In [167]:
def run_episode(env,agent,verbose = 1):

    s = env.reset()
    agent.reset_memory()
    max_step = env.n_stops
    episode_reward = 0
    
    i = 0
    while i < max_step:

        # Remember the states
        agent.remember_state(s)

        # Choose an action
        a = agent.act(s)
        
        # Take the action, and get the reward from environment
        s_next,r,done = env.step(a)

        # Tweak the reward
        r = -1 * r
        
        if verbose: print(s_next,r,done)
        
        # Update our knowledge in the Q-table
        agent.train(s,a,r,s_next)
        
        # Update the caches
        episode_reward += r
        s = s_next
        
        # If the episode is terminated
        i += 1
        if done:
            break
            
    return env,agent,episode_reward

Best solution so far with parameters alpha:0.012, gamma:0.4, is 4.0% slower than google's solution